In [1]:
# Final processing of the pipeline
import pandas as pd
data = pd.read_csv('hatefulmemes.csv')

import os
import glob

def get_image_paths(directory):
    # Define the patterns for the file types we're interested in
    patterns = ["*.jpg", "*.jpeg", "*.png"]

    # Use a list comprehension to get a list of all image paths
    image_paths = [path.split("\\")[-1] for pattern in patterns for path in glob.glob(os.path.join(directory, pattern))]

    return image_paths

# Assume `directory` is the path to your directory of images
image_paths = get_image_paths('D:/ASU/Data Mining/Project experimentation/Mask-RCNN-TF2/all_images/')
data['img'] = data['img'].apply(lambda x: x.split('/')[-1])

data = data[data['img'].isin(image_paths)]
data = data.iloc[:1000,:]
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from mrcnn import utils
from mrcnn.config import Config
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
import numpy as np
from multiprocessing import Pool
from tqdm.contrib.concurrent import process_map
import cv2
import os
import cv2
import numpy as np
from mrcnn.config import Config
from mrcnn import model as modellib, utils
import pickle
from tqdm import tqdm
tqdm.pandas()
class InferenceConfig(Config):
    NAME = "coco_inference"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 80  # COCO dataset has 80 classes + one background class
    IMAGE_MIN_DIM = 800
    IMAGE_MAX_DIM = 1024

config = InferenceConfig()

def load_and_preprocess_image(image_path):
    # Load the image
    
    image = cv2.imread(f"D:/ASU/Data Mining/Project experimentation/Mask-RCNN-TF2/all_images//{image_path.split('/')[-1]}")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Resize the image
    image, window, scale, padding, crop = utils.resize_image(
        image,
        min_dim=config.IMAGE_MIN_DIM,
        min_scale=config.IMAGE_MIN_SCALE,
        max_dim=config.IMAGE_MAX_DIM,
        mode=config.IMAGE_RESIZE_MODE)

    # Mold the image (subtract the mean pixel value)
    molded_image = modellib.mold_image(image, config)

    # Add dimension
    molded_image = np.expand_dims(molded_image, 0)
    # Create the image metadata
    image_meta = modellib.compose_image_meta(
        image_id=0, 
        original_image_shape=image.shape, 
        image_shape=image.shape, 
        window=window, 
        scale=scale, 
        active_class_ids=np.zeros([config.NUM_CLASSES], dtype=np.int32))

    # Add dimension
    image_meta = np.expand_dims(image_meta, 0)
    backbone_shapes = modellib.compute_backbone_shapes(config, config.IMAGE_SHAPE)
    anchors = utils.generate_pyramid_anchors(config.RPN_ANCHOR_SCALES, 
                                            config.RPN_ANCHOR_RATIOS,
                                            backbone_shapes,
                                            config.BACKBONE_STRIDES, 
                                            config.RPN_ANCHOR_STRIDE)

    # Match the anchors to the image shape
    anchors = np.broadcast_to(anchors, (config.BATCH_SIZE,) + anchors.shape)

    return molded_image, image_meta, anchors


def clean_text_with_nltk(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Tokenize the text into words
    words = word_tokenize(text)
    
    # Remove punctuation and numbers
    words = [word for word in words if word.isalpha()]
    
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    
    # Join the cleaned words back into a string
    cleaned_text = ' '.join(words)
    
    return cleaned_text

data["clean_text"] = data["text"].apply(clean_text_with_nltk)

from transformers import BertTokenizer, TFBertModel
from keras.layers import Input, Dense
from keras.models import Model

model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertModel.from_pretrained(model_name)

# Set all layers of BERT to not trainable
for layer in model.layers:
    
    layer.trainable = False

def get_text_embedding(text, model, tokenizer):
    text_tokens = tokenizer(text, padding='max_length', truncation=True, return_tensors="tf", max_length=128)
    return np.reshape(model(text_tokens['input_ids'])[0][:, 0, :].numpy(), (768))

def preprocessing_image(image_path):
    molded_image, image_meta, anchors = load_and_preprocess_image(image_path.split('\\')[-1])
    return molded_image, image_meta, anchors

def preprocess_text(cleaned_text):
    return get_text_embedding(cleaned_text, model=model, tokenizer=tokenizer)

def data_generator(df, batch_size):
    while True:
        for i in range(0, len(df), batch_size):
            batch_df = df.iloc[i:i+batch_size]
            
            molded_images = []
            image_metadatas = []
            anchors = []
            text_data = []
            
            for _, row in batch_df.iterrows():
                molded_image, image_metadata, anchor = preprocessing_image(row["img"])
                text_datum = preprocess_text(row["clean_text"])
                
                molded_images.append(molded_image[0])
                image_metadatas.append(image_metadata[0])
                anchors.append(anchor)
                text_data.append(text_datum)
            
            molded_images = np.array(molded_images)
            image_metadatas = np.array(image_metadatas)
            anchors = np.squeeze(np.array(anchors), axis=1)
            text_data = np.array(text_data)
            yield [molded_images, image_metadatas, anchors, text_data], batch_df['label'].values

Using TensorFlow backend.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [2]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)
test_generator = data_generator(test_df, batch_size=1)
train_generator = data_generator(train_df, batch_size=1)

In [3]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
mrcnn_model = modellib.MaskRCNN(mode="inference", model_dir=os.getcwd(), config=config)
mrcnn_model.load_weights('mask_rcnn_coco.h5', by_name=True)
# Print the names of all layers in the model
# Freeze the layers of the Mask R-CNN model
for layer in mrcnn_model.keras_model.layers:
    layer.trainable = False
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Dense, Concatenate, Input

# Assume `model` is your original Mask R-CNN model

# Get the output of the FPN layers
fpn_layer_names = ['fpn_p2', 'fpn_p3', 'fpn_p4', 'fpn_p5', 'fpn_p6']
fpn_outputs = [mrcnn_model.keras_model.get_layer(name).output for name in fpn_layer_names]

# Apply Global Average Pooling to each FPN output
pooled_outputs = [GlobalAveragePooling2D()(output) for output in fpn_outputs]

# Concatenate the pooled outputs
concatenated_outputs = Concatenate()(pooled_outputs)

# Project the concatenated outputs into a 768-dimensional vector
outputs = Dense(768)(concatenated_outputs)

text_inputs = Input(shape=(768,)
)

cancatenation_image_text = Concatenate()([outputs, text_inputs])
binary_classification = Dense(1, activation='sigmoid')(cancatenation_image_text)

# Create the final model



image_model = Model(inputs=mrcnn_model.keras_model.input+[text_inputs], outputs=binary_classification)
image_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [5]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

myconfig = ConfigProto()
myconfig.gpu_options.allow_growth = True
session = InteractiveSession(config=myconfig)
session.as_default()

In [7]:
validation_steps = len(test_df) // 1

history = image_model.fit(
    x=train_generator,
    steps_per_epoch=len(train_df) // 1,
    epochs=10,
    validation_data=test_generator,
    validation_steps=validation_steps
)

Epoch 1/10
800/800 [==============================] - 1484s 2s/step - loss: 2.8605 - accuracy: 0.5888 - val_loss: 1.2875 - val_accuracy: 0.6100
Epoch 2/10
800/800 [==============================] - 1467s 2s/step - loss: 0.8138 - accuracy: 0.6100 - val_loss: 0.9700 - val_accuracy: 0.6200
Epoch 3/10
800/800 [==============================] - 1471s 2s/step - loss: 0.8449 - accuracy: 0.5975 - val_loss: 1.9617 - val_accuracy: 0.6200
Epoch 4/10
800/800 [==============================] - 1471s 2s/step - loss: 1.1302 - accuracy: 0.6050 - val_loss: 0.7012 - val_accuracy: 0.6500
Epoch 5/10
800/800 [==============================] - 1479s 2s/step - loss: 1.0170 - accuracy: 0.6025 - val_loss: 4.8544 - val_accuracy: 0.6150
Epoch 6/10
800/800 [==============================] - 1510s 2s/step - loss: 1.3166 - accuracy: 0.6075 - val_loss: 0.9550 - val_accuracy: 0.5650
Epoch 7/10
800/800 [==============================] - 1451s 2s/step - loss: 0.9527 - accuracy: 0.6325 - val_loss: 0.7145 - val_accuracy:

In [ ]:
from transformers import ViLBERTTokenizer, ViLBERTModel

# Load pre-trained model tokenizer
tokenizer = ViLBERTTokenizer.from_pretrained('bert-base-uncased')

# Load pre-trained model
model = ViLBERTModel.from_pretrained('bert-base-uncased')

In [6]:
for layer in image_model.layers:
    if layer.trainable:
        print(f"Layer {layer.name} is trainable")

Layer global_average_pooling2d_1 is trainable
Layer global_average_pooling2d_2 is trainable
Layer global_average_pooling2d_3 is trainable
Layer global_average_pooling2d_4 is trainable
Layer global_average_pooling2d_5 is trainable
Layer concatenate_1 is trainable
Layer dense_3 is trainable
Layer concatenate_2 is trainable
Layer dense_4 is trainable
